In [2]:
%cd ..

/home/adam/Projects/jointformer


In [3]:
import random

from jointformer.utils.datasets.smiles.base import SMILESDataset
from jointformer.utils.transforms.permute import PermuteSMILES
from jointformer.utils.transforms.smiles_enumerator import SmilesEnumerator
from jointformer.utils.tokenizers.smiles_tokenizers.regex import SMILESTokenizer
from jointformer.models.base import Transformer

import torchvision.transforms as transforms

%load_ext autoreload
%autoreload 2

In [6]:
# Data

PATH_TO_TXT_DATA = 'data/guacamol/test/smiles.txt'
PATH_TO_VOCAB = 'jointformer/utils/tokenizers/smiles_tokenizers/molgpt_vocabulary.txt'
NUM_SAMPLES = 1000
MAX_LENGTH = 128
VALIDATE = False

In [7]:
# Dataset and Tokenizer

transform = transforms.Compose([
    SmilesEnumerator(),
])

dataset = SMILESDataset(file_path=PATH_TO_TXT_DATA, num_samples=NUM_SAMPLES, validate=VALIDATE, transform=transform)
tokenizer = SMILESTokenizer(path_to_vocabulary=PATH_TO_VOCAB, max_length=128)

In [8]:
bs = 2
smiles = [dataset[i] for i in range(10)]
inputs = tokenizer(smiles)




TypeError: expected string or bytes-like object

In [40]:
tokenizer = SmilesTokenizer('data/vocabularies/deepchem.txt', max_length=128)

print(tokenizer.encode("CC(=O)OC1=CC=CC=C1C(=O)O"))

[12, 16, 16, 17, 22, 19, 18, 19, 16, 20, 22, 16, 16, 22, 16, 16, 22, 16, 20, 16, 17, 22, 19, 18, 19, 13]
